1.IMPORTING NECESSARY LIBRARIES

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

2.DATASET PREPARATION

In [ ]:
import zipfile
import os

zip_path = '/content/dataset.zip'  # Update if your zip file is located elsewhere
extract_to = 'dataset'

# Extract the dataset if everything is okay
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset extracted to:", extract_to)

Dataset extracted to: dataset


3.DATA AUGMENTATION AND PREPROCESSING AND LOADING PRETRAINED MODELS

In [ ]:
# Set image size and parameters
img_size = (224, 224)
batch_size = 32
num_classes = 3  # Benign and malignant
# Step 1: Data augmentation and preprocessing for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test data
train_generator = train_datagen.flow_from_directory(
    '/content/dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = valid_test_datagen.flow_from_directory(
    '/content/dataset',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)
# Step 2: Load your saved ResNet and VGG models
resnet = load_model('/content/resnet_model.h5')
vgg = load_model('/content/vgg_19_model.h5')

Found 1312 images belonging to 1 classes.
Found 1312 images belonging to 1 classes.


4.FEATURE EXTRACTION FROM PRETRAINED MODELS

5.CREATING A SHARED INPUT LAYER

6.COMBING FEATURES AND ADDING CUSTOM CLASSIFICATION LAYERS

7.DEFINING AND COMPILING THE MODEL

8.TRAINING THE MODEL

9.EVALUATING THE MODEL

In [ ]:
# Step 3: Remove the top layers (the classification head) to get the features
def remove_top(model):
    model.layers[-1].activation = None
    return models.Model(inputs=model.input, outputs=model.layers[-2].output)

resnet_features = remove_top(resnet)
vgg_features = remove_top(vgg)

# Step 4: Create a shared input layer
input_layer = layers.Input(shape=(224, 224, 3))

# Pass the same input through both models
resnet_output = resnet_features(input_layer)
vgg_output = vgg_features(input_layer)

# Step 5: Combine the feature outputs from ResNet and VGG
combined = layers.concatenate([resnet_output, vgg_output])

# Step 6: Add new fully connected layers for classification
x = layers.Dense(512, activation='relu')(combined)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(3, activation='softmax')(x)  # Ensure 3 classes

# Step 7: Define the new model
ensemble_model = models.Model(inputs=input_layer, outputs=predictions)

# Step 8: Compile the model
ensemble_model.compile(optimizer=Adam(learning_rate=1e-5),
                       loss='sparse_categorical_crossentropy',  # Ensure this matches your label encoding
                       metrics=['accuracy'])

# Step 9: Train the model
history = ensemble_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    epochs=10
)
